# Hypothesis Testing
Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available :
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    University_towns = pd.read_csv("university_towns.txt",sep="\n",header=None)
    result=[]
    final_result=[]
    state_name=None
    for index,value in University_towns.iterrows():
        region_name = None
        string = value[0]
        if string[-6:] =='[edit]':
            string = string[:-6]
            state_name = string
            continue
        if '(' in string:
            region_name = string[:string.index('(')-1]
            result.append([state_name,region_name])
        else:
            region_name = string
            result.append([state_name,region_name])
        final_result.append(string)
    df = pd.DataFrame(result,columns = ['State','RegionName'])
    return df

get_list_of_university_towns()

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls", skiprows=219)
    df = df[['1999q4',9926.1]]
    df.columns = ['Quarter','GDP']
    temp=None
    for i in range(2,len(df)):
        if ((df.iloc[i-2][1] > df.iloc[i-1][1]) and (df.iloc[i-1][1] > df.iloc[i][1])):
            temp=df.iloc[i-2][0]
            return temp
get_recession_start()

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls", skiprows=219)
    df = df[['1999q4',9926.1]]
    df.columns = ['Quarter','GDP']
    temp=None
    rec_start =get_recession_start()
    start_index = df[df['Quarter']==rec_start].index.tolist()[0]
    df = df.iloc[start_index:]
    for i in range(2,len(df)):
        if ((df.iloc[i-2][1] < df.iloc[i-1][1]) and (df.iloc[i-1][1] < df.iloc[i][1])):
            temp=df.iloc[i][0]
            return temp
get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = pd.read_excel("gdplev.xls", skiprows=219)
    df = df[['1999q4',9926.1]]
    df.columns = ['Quarter','GDP']
    temp=None
    rec_start =get_recession_start()
    start_index = df[df['Quarter']==rec_start].index.tolist()[0]
    rec_end = get_recession_end()
    end_index = df[df['Quarter']==rec_end].index.tolist()[0]
    df = df.iloc[start_index : end_index+1]
    rec_bottom = df[df['GDP'] == df['GDP'].min()].index.tolist()[0]-start_index
    bottom_quarter = df.iloc[rec_bottom][0]
    temp = bottom_quarter
    
    return temp
get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv("City_Zhvi_AllHomes.csv")
    df = df.drop(['RegionID','Metro','CountyName','SizeRank'], axis=1)
    for i in range(1996,2000):
        for j in range(1,13):
            if j<10:
                column_name = str(str(i)+'-0'+str(j))
            else:
                column_name = str(str(i)+'-'+str(j))
            if column_name in ['1996-01','1996-02','1996-03']:
                continue
            else:
                df = df.drop([column_name],axis=1)
    for i in range(2000,2016):
        df[str(i)+'q1'] = df[[str(i)+'-01',str(i)+'-02',str(i)+'-03']].mean(1)
        df[str(i)+'q2'] = df[[str(i)+'-04',str(i)+'-05',str(i)+'-06']].mean(1)
        df[str(i)+'q3'] = df[[str(i)+'-07',str(i)+'-08',str(i)+'-09']].mean(1)
        df[str(i)+'q4'] = df[[str(i)+'-10',str(i)+'-11',str(i)+'-12']].mean(1)
        df = df.drop([str(i)+'-01',str(i)+'-02',str(i)+'-03',str(i)+'-04',str(i)+'-05',str(i)+'-06',str(i)+'-07',str(i)+'-08',str(i)+'-09',str(i)+'-10',str(i)+'-11',str(i)+'-12'], axis=1)
    i=2016
    df[str(i)+'q1'] = df[[str(i)+'-01',str(i)+'-02',str(i)+'-03']].mean(1)
    df[str(i)+'q2'] = df[[str(i)+'-04',str(i)+'-05',str(i)+'-06']].mean(1)
    df[str(i)+'q3'] = df[[str(i)+'-07',str(i)+'-08']].mean(1)
    df = df.drop([str(i)+'-01',str(i)+'-02',str(i)+'-03',str(i)+'-04',str(i)+'-05',str(i)+'-06',str(i)+'-07',str(i)+'-08'], axis=1)
    df = df.replace({'State':states})
    df = df.set_index(['State','RegionName'])
    return df
convert_housing_data_to_quarters()

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    rec_start = get_recession_start()
    rec_end = get_recession_end()
    rec_bottom = get_recession_bottom()
    df = convert_housing_data_to_quarters()
    uni = get_list_of_university_towns()
    bstart = df.columns[df.columns.get_loc(rec_start)-1]
    df['ratio'] = df[rec_bottom] - df[bstart]
    df = df[[rec_bottom,bstart,'ratio']]
    df = df.reset_index()
    unidata = df.merge(uni,how="inner" ,on =['State','RegionName'])
    unidata['uni'] = True
    new_df = pd.merge(df,unidata,how="outer", on=['State','RegionName',rec_bottom,bstart,'ratio'])
    new_df['uni'] = new_df['uni'].fillna(False)
    university_towns = new_df[new_df['uni']==True]
    non_university_towns = new_df[new_df['uni'] == False]
    t,p = ttest_ind(university_towns['ratio'].dropna() , non_university_towns['ratio'].dropna())
    different = True if p < 0.01 else False
    better = "university town" if university_towns['ratio'].mean() > non_university_towns['ratio'].mean() else "non university town"
    temp = (different,p,better)
    return temp
run_ttest()